***

<div align="center">
  <h3> Desarrollo de un gestor de inteligencia colectiva y un marco de trabajo para el aprendizaje a través de la resolución colaborativa de      problemas en entornos educativos transmediales </h3>
  <h4> Reporte técnico de Análisis de Datos. </h4>
  <h5> Análisis de Matriz de frecuencias tipo de eventos ejecutados por un usuario en GIANT </h5>
  <h3> RT - 009 - 1 </h3>
  <h3> JOHN FABER FLÓREZ VASCO - MÓDULO DE DATOS - GIANT </h3>
</div>

***

<p style='text-align: justify;'> 

<div align="center">
  <h2> 1. IMPORTANDO PAQUETES Y CONFIGURACIÓN DEL ENTORNO  </h2>
</div>

</p>

In [1]:
import Pkg; 
Pkg.add("Pkg")
using Pkg;

   Updating registry at `~/.julia/registries/General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [2]:
# Cargando los paquetes instalados
using DataFrames
using CSV
using Plots
using Statistics
using StatsPlots
using StatsBase
using Lathe
using Clustering #Métodos para la agrupación de datos y evaluación de la calidad de la agrupación.
using CSV
using Distances
gr()

Plots.GRBackend()

<div align="center">
  <h2> 2. CARGANDO LOS DATOS  </h2>
</div>

In [107]:
# Se van a leer los datos usando un Archivo tipo CSV y lo vamos a convertir en un DataFrame 
eventsDataFrame = DataFrame(CSV.File("events_giant_2.csv"))
first(eventsDataFrame,5)

,_id,id,eventname
,String,Int64,String
1,5f61558294d5986a8766f0ec,1,\\giant\\event\\extended_challenge_team_viewed
2,5f61558294d5986a8766f0ed,2,\\core\\event\\user_login_failed
3,5f61558294d5986a8766f0ee,3,\\core\\event\\user_loggedin
4,5f61558294d5986a8766f0ef,4,\\core\\event\\dashboard_viewed
5,5f61558294d5986a8766f0f0,5,\\core\\event\\course_viewed


In [108]:
#Resumen del marco de los datos
println(size(eventsDataFrame))

(20182, 9)


In [109]:
#Convirtiendo variables en tipo String
eventsDataFrame[!,[:eventname, :component, :action, :target, :crud]]=convert(Array{String}, eventsDataFrame[!,[:eventname, :component, :action, :target, :crud]])

20182×5 Array{String,2}:
 "\\giant\\event\\extended_challenge_team_viewed"  …  "extended_challenge_team"  "r"
 "\\core\\event\\user_login_failed"                   "user_login"               "r"
 "\\core\\event\\user_loggedin"                       "user"                     "r"
 "\\core\\event\\dashboard_viewed"                    "dashboard"                "r"
 "\\core\\event\\course_viewed"                       "course"                   "r"
 "\\core\\event\\course_viewed"                    …  "course"                   "r"
 "\\core\\event\\dashboard_viewed"                    "dashboard"                "r"
 "\\core\\event\\dashboard_viewed"                    "dashboard"                "r"
 "\\core\\event\\user_loggedin"                       "user"                     "r"
 "\\core\\event\\dashboard_viewed"                    "dashboard"                "r"
 "\\core\\event\\user_loggedin"                    …  "user"                     "r"
 "\\core\\event\\dashboard_viewed"      

In [110]:
#IDENTIFICANDO CATEGORIAS de la variable eventname
l = unique(eventsDataFrame[!,:eventname])

100-element Array{String,1}:
 "\\giant\\event\\extended_challenge_team_viewed"
 "\\core\\event\\user_login_failed"
 "\\core\\event\\user_loggedin"
 "\\core\\event\\dashboard_viewed"
 "\\core\\event\\course_viewed"
 "\\core\\event\\user_profile_viewed"
 "\\mod_scorm\\event\\course_module_viewed"
 "\\mod_scorm\\event\\sco_launched"
 "\\giant\\event\\collaborative_wall_viewed"
 "\\mod_lesson\\event\\lesson_started"
 "\\mod_lesson\\event\\course_module_viewed"
 "\\mod_lesson\\event\\content_page_viewed"
 "\\core_h5p\\event\\h5p_viewed"
 ⋮
 "\\mod_assign\\event\\grading_table_viewed"
 "\\mod_resource\\event\\course_module_viewed"
 "\\assignsubmission_comments\\event\\comment_created"
 "\\core\\event\\calendar_event_deleted"
 "\\mod_assign\\event\\grading_form_viewed"
 "\\mod_assign\\event\\remove_submission_form_viewed"
 "\\mod_assign\\event\\submission_status_updated"
 "\\core\\event\\course_created"
 "\\core\\event\\course_section_created"
 "\\core\\event\\enrol_instance_created"
 "\\tool

In [112]:
dfEventsGiant = DataFrame(EVENT_TYPE = l )

,EVENT_TYPE
,String
1,\\giant\\event\\extended_challenge_team_viewed
2,\\core\\event\\user_login_failed
3,\\core\\event\\user_loggedin
4,\\core\\event\\dashboard_viewed
5,\\core\\event\\course_viewed
6,\\core\\event\\user_profile_viewed
7,\\mod_scorm\\event\\course_module_viewed
8,\\mod_scorm\\event\\sco_launched
9,\\giant\\event\\collaborative_wall_viewed


In [113]:
CSV.write("events_giant.csv", dfEventsGiant)

"events_giant.csv"

In [24]:
#Cambiando valores categóricos por numéricos en la columna EventName
eventsDataFrame[!,:eventname]=replace(eventsDataFrame[!,:eventname], Pair.(l, axes(l, 1))...)

20182-element Array{Any,1}:
  1
  2
  3
  4
  5
  5
  4
  4
  3
  4
  3
  4
  6
  ⋮
 10
 11
 12
  5
  5
 11
 40
 12
 13
 11
 12
 13

In [25]:
maximum(eventsDataFrame[!,:eventname])

100

In [58]:
dfUserEvents = DataFrame(USER_ID = eventsDataFrame[!,:userid], EVENT_TYPE = convert(Array{Int64}, eventsDataFrame[!,:eventname]) )

,USER_ID,EVENT_TYPE
,Int64,Int64
1,78,1
2,141,2
3,141,3
4,141,4
5,141,5
6,141,5
7,141,4
8,141,4
9,48,3


In [27]:
maximum(dfUserEvents[!,:EVENT_TYPE])

100

In [28]:
scatter(dfUserEvents.USER_ID, dfUserEvents.EVENT_TYPE, marker_z=dfUserEvents.EVENT_TYPE,
        color=:lightrainbow,title="Eventos ejecutados por usuario",ylabel ="Evento ejecutado",xlabel="Identificador del Usuario",legend=false,size=(700,300))

<div align="center">
  <h2> 3. PREPROCESAMIENTO DE LOS DATOS </h2>
</div>
<h3> 3.1 One Hot Encoding </h3>
<p style='text-align: justify;'> 
    Vamos a convertir el tipo de evento para lograr formar múltiples columnas numéricas - (normalización de los datos) ya que tenemos estás categorías. Esto nos va a ayudar para que las variables puedan introducirse en los algoritmos de aprendizaje automático para hacer un mejor trabajo en la predicción.
</p>

In [29]:
scaled_feature = Lathe.preprocess.OneHotEncode(dfUserEvents,:EVENT_TYPE)

(::Lathe.preprocess.var"#predict#23") (generic function with 1 method)

In [30]:
EventsMatrix=select!(dfUserEvents, Not(:EVENT_TYPE))

,USER_ID,1,2,3,4,5,6,7,8,9,10,11,12
,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool
1,78,1,0,0,0,0,0,0,0,0,0,0,0
2,141,0,1,0,0,0,0,0,0,0,0,0,0
3,141,0,0,1,0,0,0,0,0,0,0,0,0
4,141,0,0,0,1,0,0,0,0,0,0,0,0
5,141,0,0,0,0,1,0,0,0,0,0,0,0
6,141,0,0,0,0,1,0,0,0,0,0,0,0
7,141,0,0,0,1,0,0,0,0,0,0,0,0
8,141,0,0,0,1,0,0,0,0,0,0,0,0
9,48,0,0,1,0,0,0,0,0,0,0,0,0


In [31]:
#Obteniendo un arreglo de los eventos realizados por un usuario
to_group_events = names(EventsMatrix[!, Not(:USER_ID)])

100-element Array{String,1}:
 "1"
 "2"
 "3"
 "4"
 "5"
 "6"
 "7"
 "8"
 "9"
 "10"
 "11"
 "12"
 "13"
 ⋮
 "89"
 "90"
 "91"
 "92"
 "93"
 "94"
 "95"
 "96"
 "97"
 "98"
 "99"
 "100"

In [34]:
#Agrupando un único usuario con la suma de los eventos realizados
frequencyEventsMatrix=combine(groupby(EventsMatrix, :USER_ID), to_group_events .=> sum .=> to_group_events)

,USER_ID,1,2,3,4,5,6,7,8,9,10,11
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,78,6,0,3,3,8,0,6,6,1,0,0
2,141,0,3,16,23,131,0,33,30,6,3,89
3,48,0,2,21,28,123,1,35,32,29,2,30
4,139,0,1,6,6,19,0,4,2,0,2,28
5,63,0,0,5,6,26,0,19,10,5,0,0
6,57,0,0,3,3,18,0,12,9,5,0,0
7,99,0,0,4,4,18,0,9,7,5,0,0
8,0,0,45,0,0,0,0,0,0,0,0,0
9,142,2,0,3,8,94,1,78,48,9,2,12


In [74]:
#Organizando la matriz de frecuencia de los eventos por identificador de usuario
frequencyEventsMatrix=sort!(frequencyEventsMatrix, [:USER_ID])

,USER_ID,1,2,3,4,5,6,7,8,9,10,11
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,-10,0,0,0,0,0,0,0,0,0,0,0
2,-1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,45,0,0,0,0,0,0,0,0,0
4,2,0,3,39,80,126,10,4,0,0,0,74
5,3,0,1,6,7,26,1,5,1,0,0,0
6,4,3,2,34,52,205,10,100,53,15,0,19
7,6,0,0,3,3,39,0,11,9,0,0,3
8,8,2,0,1,1,2,0,1,1,0,0,0
9,9,11,0,8,15,36,30,3,1,0,0,70


In [36]:
to_group_users = frequencyEventsMatrix[!,:USER_ID]

88-element Array{Int64,1}:
 -10
  -1
   0
   2
   3
   4
   6
   8
   9
  10
  18
  19
  23
   ⋮
 125
 128
 132
 133
 134
 137
 139
 140
 141
 142
 144
 145

In [37]:
describe(frequencyEventsMatrix)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Nothing,Nothing,DataType
1,USER_ID,72.4318,-10,73.5,145,,,Int64
2,1,3.04545,0,0.0,134,,,Int64
3,2,2.35227,0,1.0,45,,,Int64
4,3,8.71591,0,5.0,48,,,Int64
5,4,11.1705,0,7.0,80,,,Int64
6,5,52.3977,0,27.0,310,,,Int64
7,6,1.82955,0,0.0,45,,,Int64
8,7,18.5341,0,11.5,144,,,Int64
9,8,11.6591,0,9.0,71,,,Int64


In [38]:
frequencyEventsDataFrameWithoutUsers=frequencyEventsMatrix[!, Not(:USER_ID)]

,1,2,3,4,5,6,7,8,9,10,11,12
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,45,0,0,0,0,0,0,0,0,0,0
4,0,3,39,80,126,10,4,0,0,0,74,74
5,0,1,6,7,26,1,5,1,0,0,0,0
6,3,2,34,52,205,10,100,53,15,0,19,19
7,0,0,3,3,39,0,11,9,0,0,3,3
8,2,0,1,1,2,0,1,1,0,0,0,0
9,11,0,8,15,36,30,3,1,0,0,70,70


<div align="center">
  <h2> 4. NORMALIZACIÓN DE LOS DATOS </h2>
</div>

In [44]:
frequencyMatrixEvents=Matrix(convert(Array{Float64}, frequencyEventsDataFrameWithoutUsers))

88×100 Array{Float64,2}:
   0.0   0.0   0.0   0.0    0.0   0.0  …  0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0   0.0   0.0   0.0    0.0   0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0  45.0   0.0   0.0    0.0   0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0   3.0  39.0  80.0  126.0  10.0     0.0  0.0  1.0  10.0  3.0  1.0  1.0
   0.0   1.0   6.0   7.0   26.0   1.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
   3.0   2.0  34.0  52.0  205.0  10.0  …  0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0   0.0   3.0   3.0   39.0   0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
   2.0   0.0   1.0   1.0    2.0   0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
  11.0   0.0   8.0  15.0   36.0  30.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
 134.0   6.0  48.0  70.0  310.0  45.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0   3.0   5.0   8.0   95.0   0.0  …  0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0   0.0  11.0   7.0  109.0   0.0     0.0  0.0  0.0   0.0  0.0  0.0  0.0
   0.0   6.0   3.0   4.0   15.0   0.0     0.0  0.0 

In [45]:
dt = fit(ZScoreTransform, Matrix(frequencyMatrixEvents), dims=2)

ZScoreTransform{Float64}(88, 2, [0.16, 1.59, 0.77, 9.85, 0.73, 5.78, 0.74, 0.08, 3.0, 11.11  …  4.9, 5.75, 0.2, 5.38, 1.18, 1.53, 5.65, 3.58, 0.13, 0.1], [1.5999999999999994, 15.89999999999999, 5.00273662482584, 21.37182843149014, 2.8634054740973975, 23.95643352151374, 4.155123404310707, 0.3387439663632019, 11.779969302492974, 38.084142621200265  …  14.651162604263341, 18.10819056826401, 0.8645662192537673, 17.573384374782183, 4.639270231709766, 5.236729259120951, 19.329871685915034, 13.194106428418873, 0.7608474807008884, 0.5025189076296075])

In [87]:
normalizeFrequencyMatrixEvents=StatsBase.transform(dt, Matrix(frequencyMatrixEvents))

88×100 Array{Float64,2}:
 -0.1       -0.1        -0.1       …  -0.1       -0.1       -0.1
 -0.1       -0.1        -0.1          -0.1       -0.1       -0.1
 -0.153916   8.84116    -0.153916     -0.153916  -0.153916  -0.153916
 -0.460887  -0.320515    1.36395      -0.320515  -0.414097  -0.414097
 -0.254941   0.0942933   1.84047      -0.254941  -0.254941  -0.254941
 -0.116044  -0.157786    1.17797   …  -0.241271  -0.241271  -0.241271
 -0.178093  -0.178093    0.543907     -0.178093  -0.178093  -0.178093
  5.668     -0.236167    2.71592      -0.236167  -0.236167  -0.236167
  0.679119  -0.25467     0.424449     -0.25467   -0.25467   -0.25467
  3.2268    -0.134177    0.968645     -0.291722  -0.291722  -0.291722
 -0.203365   0.0885467   0.283155  …  -0.203365  -0.203365  -0.203365
 -0.314579  -0.314579    0.372002     -0.314579  -0.314579  -0.314579
 -0.239758   2.95701     1.35863      -0.239758  -0.239758  -0.239758
  ⋮                                ⋱                        
 -0.332438  -0.

<div align="center">
  <h2> 5. OBSERVACIÓN DE RESULTADOS APLICANDO APRENDIZAJE AUTOMÁTICO NO SUPERVISADO </h2>
</div>

<div align="center">
  <h2> 5.1 K-MEANS </h2>
</div>

In [99]:
#K-MEANS
C = kmeans(Matrix(normalizeFrequencyMatrixEvents)', 4)

KmeansResult{Array{Float64,2},Float64,Int64}([0.7316732319400325 -0.21348864167498285 0.5844143189202645 -0.2838697363129051; 0.18794713793931267 -0.10973930224610051 2.6558346488466777 -0.03544015992824653; … ; -0.24238039539708447 -0.31508169094701377 -0.1667315746113436 -0.30621056639044875; -0.24238039539708447 -0.31508169094701377 -0.1667315746113436 -0.30621056639044875], [3, 3, 3, 2, 1, 1, 1, 1, 2, 1  …  2, 2, 3, 2, 2, 1, 2, 1, 3, 3], [114.54292816319244, 113.94201144836728, 76.4416160750558, 51.08647453631994, 23.543474607492016, 3.747902538962961, 9.771900834446, 33.00828359690789, 29.15213752071665, 11.395141966896688  …  6.856492944741888, 8.618539294815236, 65.05744742024643, 9.29227817958079, 14.781089549558658, 16.68742835566883, 4.713488908629671, 13.97278806730722, 39.6199079062183, 38.962762310896764], [35, 27, 12, 14], [35, 27, 12, 14], 1789.2736203006016, 6, true)

In [ ]:
frequencyEventsKmeans=frequencyEventsMatrix

In [100]:
insertcols!(frequencyEventsKmeans,2,:kmeans_4=>C.assignments, makeunique=true)

,USER_ID,kmeans_4_1,kmeans_4,kmedoids4,1,2,3,4,5,6
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3,3,1,1,0,1,6,7,26,1
2,6,3,1,1,0,0,3,3,39,0
3,8,3,1,1,2,0,1,1,2,0
4,23,2,1,1,0,6,3,4,15,0
5,28,1,1,1,0,1,3,3,35,0
6,29,1,1,1,0,0,1,2,3,0
7,36,1,1,1,0,0,1,2,16,1
8,42,1,1,1,0,3,6,8,29,0
9,60,2,1,1,0,0,1,1,5,0


In [101]:
select!(frequencyEventsKmeans, Not(:kmeans_4))

,USER_ID,kmeans_4_1,kmedoids4,1,2,3,4,5,6,7
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3,3,1,0,1,6,7,26,1,5
2,6,3,1,0,0,3,3,39,0,11
3,8,3,1,2,0,1,1,2,0,1
4,23,2,1,0,6,3,4,15,0,6
5,28,1,1,0,1,3,3,35,0,17
6,29,1,1,0,0,1,2,3,0,0
7,36,1,1,0,0,1,2,16,1,6
8,42,1,1,0,3,6,8,29,0,12
9,60,2,1,0,0,1,1,5,0,3


In [103]:
gdKmeans=groupby(frequencyEventsKmeans, :kmeans_4_1)

,USER_ID,kmeans_4_1,kmedoids4,1,2,3,4,5,6,7
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3,3,1,0,1,6,7,26,1,5
2,6,3,1,0,0,3,3,39,0,11
3,8,3,1,2,0,1,1,2,0,1
4,-1,3,1,0,0,0,0,0,0,0
5,128,3,1,0,3,0,0,0,0,0
6,145,3,1,0,1,2,4,0,0,0
7,106,3,3,0,3,22,27,142,0,29
8,112,3,3,0,2,41,58,208,10,80
9,92,3,4,0,0,9,12,64,3,15


In [85]:
CSV.write("groups_events_giant_2_4.csv", gdKmeans[4])

"groups_events_giant_2_4.csv"

<div align="center">
  <h2> 5.2 K-MEDOIDS </h2>
</div>

In [88]:
xmatrix = Matrix(normalizeFrequencyMatrixEvents)'
D = pairwise(Euclidean(), xmatrix, xmatrix, dims=2)
K = kmedoids(D, 4)

KmedoidsResult{Float64}([37, 69, 62, 6], [2, 1, 2, 3, 4, 4, 4, 4, 3, 4  …  3, 3, 4, 3, 3, 4, 3, 4, 2, 2], [14.166657660388402, 14.081077110803571, 6.370545758349936, 7.825065397747202, 4.530660921536821, 0.0, 2.4688776810442685, 6.925931517336633, 6.041361175304278, 3.7224859111915083  …  2.868572452974013, 2.9825746283536985, 10.969272259887985, 3.3064667945688777, 4.536237045140995, 3.124125133000856, 2.9297948481698906, 3.9062517099619805, 11.479660427274199, 11.483747195658527], [17, 7, 29, 35], 386.2959021244302, 2, true)

In [89]:
insertcols!(frequencyEventsKmeans,3,:kmedoids4=>C.assignments, makeunique=true)

,USER_ID,kmeans_4,kmedoids4,1,2,3,4,5,6,7,8
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,-10,3,1,0,0,0,0,0,0,0,0
2,-1,3,1,0,0,0,0,0,0,0,0
3,0,3,1,0,45,0,0,0,0,0,0
4,2,2,3,0,3,39,80,126,10,4,0
5,3,1,1,0,1,6,7,26,1,5,1
6,4,1,3,3,2,34,52,205,10,100,53
7,6,1,1,0,0,3,3,39,0,11,9
8,8,1,1,2,0,1,1,2,0,1,1
9,9,2,2,11,0,8,15,36,30,3,1


In [104]:
frequencyEventsKmeans=sort!(frequencyEventsKmeans, [:kmeans_4_1])

,USER_ID,kmeans_4_1,kmedoids4,1,2,3,4,5,6,7
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,28,1,1,0,1,3,3,35,0,17
2,29,1,1,0,0,1,2,3,0,0
3,36,1,1,0,0,1,2,16,1,6
4,42,1,1,0,3,6,8,29,0,12
5,70,1,1,0,0,1,1,5,0,3
6,73,1,1,0,0,1,1,2,0,0
7,79,1,1,0,0,1,2,5,0,3
8,119,1,1,0,3,1,1,4,0,1
9,120,1,1,1,3,2,11,27,2,11


In [105]:
CSV.write("groups_events_giant_2_5.csv", frequencyEventsKmeans[!, Not(:kmedoids4)])

"groups_events_giant_2_5.csv"